# Iteratorパターン

* 

In [2]:
%%bash
### 変数の設定
name=Aggregate

### ソースの編集
cat <<- EOS > ${name}.java
public interface ${name} {
    public abstract Iterator iterator();
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}

In [3]:
%%bash
### 変数の設定
name=Iterator

### ソースの編集
cat <<- EOS > ${name}.java
public interface ${name} {
    public abstract boolean hasNext();
    public abstract Object next();
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}

In [4]:
%%bash
### 変数の設定
name=Book

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} {
    private String name;
    public Book(String name) {
        this.name = name;
    }
    public String getName() {
        return name;
    }
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}

In [5]:
%%bash
### 変数の設定
name=BookShelf

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} implements Aggregate {
    private Book[] books;
    private int last = 0;
    public BookShelf(int maxsize) {
        this.books = new Book[maxsize];
    }
    public Book getBookAt(int index) {
        return books(index);
    }
    public void appendBook(Book book) {
        this.book[last] = book;
        last++;
    }
    public int getLength() {
        return last;
    }
    public Iterator iterator() {
        return new BookShelfIterator(this);
    }
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}